In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [29]:
stock_data = pd.read_csv('MacroTrend_Stocks.csv', encoding='latin-1' , header=None)

financial_ratios_URLs = []
for i in range(1, len(stock_data)):
    financial_ratios_URLs.append('https://www.macrotrends.net/stocks/charts/' + str(stock_data[1][i]) + '/' + str(stock_data[0][i]) + '/financial-ratios')


In [31]:
def parse_page(driver):
    # disable all cookies
    driver.delete_all_cookies()
    # Click on the element for 40 times
    element = driver.find_element(By.ID, "jqxScrollBtnDownhorizontalScrollBarjqxgrid")
    for _ in range(50):
        element.click()

    response = driver.page_source
    soup = BeautifulSoup(response, "html.parser")
    
    header_main_div = soup.find('div', {"id": "columntablejqxgrid"})
    
    first_div = soup.find("div", {"id": "contenttablejqxgrid"})
    divs = first_div.find_all("div", {"id": lambda x: x and x.startswith("row")})
    
    page_data = []

    # Extract header data
    header_data = []
    header_divs = header_main_div.find_all("div", {"class": "jqx-grid-column-header jqx-widget-header"})
    for div in header_divs:
        cell = div.find("span", {"style": lambda x : x.startswith("text-overflow")})
        header_data.append(cell.text.strip())
    page_data.append(header_data)

    # Extract row data
    for div in divs:
        table_row = div.find_all("div", {"class": lambda x: x and x.startswith("jqx-grid-cell jqx-item")})
        row_data = [cell.text.strip() for cell in table_row]
        page_data.append(row_data)
        
    return page_data

In [33]:
for count , url in enumerate(financial_ratios_URLs):
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        # Wait for the page to load
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "main_content"))
        )
        data = []
        # Get the page source
        data += parse_page()
        df = pd.DataFrame(data)
        df.to_csv('Financial Ratios/' + str(stock_data[0][count]) + '_Financial_Ratios.csv', index=False , header=None)
        # Close the browser
        driver.quit()
    except:
        print("Error On " + str(stock_data[0][count]))
